In [ ]:
import pandas as pd
from pycaret.regression import *
from datetime import datetime

# --- Stałe ---
gender_map = {"Mężczyzna": 1, "Kobieta": 0}

# --- Funkcje pomocnicze ---
def time_to_seconds(time_str):
    try:
        h, m, s = map(int, time_str.split(':'))
        return h * 3600 + m * 60 + s
    except:
        return None

def seconds_to_hms(seconds):
    return f"{int(seconds // 3600):02}:{int((seconds % 3600) // 60):02}:{int(seconds % 60):02}"

def wczytaj_i_przetworz_plik(ścieżka, rok):
    df = pd.read_csv(ścieżka, sep=';')
    kluczowe = ['5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas', 'Czas']
    df.dropna(subset=kluczowe, inplace=True)
    
    df['Wiek'] = rok - df['Rocznik']
    df['Płeć'] = df['Płeć'].map({'K': 0, 'M': 1})
    
    df.rename(columns={
        "Czas": "Czas_półmaratonu",
        "5 km Czas": "Czas_na_5km",
        "10 km Czas": "Czas_na_10km",
        "15 km Czas": "Czas_na_15km",
        "20 km Czas": "Czas_na_20km"
    }, inplace=True)

    for kol in ['Czas_półmaratonu', 'Czas_na_5km']:
        df[kol] = df[kol].apply(time_to_seconds)

    df.drop(columns=[
        'Miejsce', 'Numer startowy', 'Imię', 'Nazwisko', 'Miasto', 'Kraj', 'Drużyna',
        'Płeć Miejsce', 'Kategoria wiekowa Miejsce', '5 km Miejsce Open',
        '10 km Miejsce Open', '15 km Miejsce Open', '20 km Miejsce Open',
        'Tempo Stabilność', 'Tempo', 'Kategoria wiekowa', 'Rocznik',
        '5 km Tempo', '10 km Tempo', '15 km Tempo', '20 km Tempo',
        '5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas'
    ], inplace=True, errors='ignore')

    return df[['Płeć', 'Wiek', 'Czas_półmaratonu', 'Czas_na_5km']]

# --- Wczytanie danych ---
df_2023 = wczytaj_i_przetworz_plik('halfmarathon_wroclaw_2023__final.csv', 2023)
df_2024 = wczytaj_i_przetworz_plik('halfmarathon_wroclaw_2024__final.csv', 2024)
df = pd.concat([df_2023, df_2024], ignore_index=True)

df

# Funkcja do trenowania modelu i przewidywania czasów na półmaratonie
def train_and_predict(target_column):
    # Ustawienie środowiska PyCaret z dodatkowymi cechami (w tym Płeć, Wiek)
    setup(data=df,
          target=target_column,
          session_id=123, 
          normalize=True,
          use_gpu=True,
          numeric_features=['Wiek', 'Płeć'],
          verbose=True
    )

    # Porównanie modeli
    best_model = compare_models(fold=5)

    # Finalizacja najlepszego modelu
    final_model = finalize_model(best_model)

    # Zapisanie najlepszego modelu
    save_model(final_model, f'{target_column}_model')

    return final_model

# Trenowanie i zapisanie modelu dla przewidywania czasu półmaratonu na podstawie czasu na 5 km
time_halfmarathon_model = train_and_predict('Czas_półmaratonu')


,Płeć,Wiek,Czas_półmaratonu,Czas_na_5km
0,1,31.0,3899,877
1,1,37.0,3983,888
2,1,27.0,4104,946
3,1,35.0,4216,971
4,1,28.0,4227,972
...,...,...,...,...
18372,0,47.0,11866,2396
18373,0,46.0,12068,2526
18374,0,69.0,12138,2960
18375,0,43.0,12512,2808
